In [3]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T


#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [4]:
dataset = Planetoid(root='/tmp/citeseer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [5]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,3694,3695,3696,3697,3698,3699,3700,3701,3702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [6]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [7]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    #print(f)
    f.clear()
    Fec.append(vec)
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0

    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [8]:
# Selective Domain Feature
feature_names=[]
for i in range(6):
    feature_names.append("{}".format(i))
x =np.array(SFec)
data1=pd.DataFrame(x,columns=feature_names)
data1.head()

,0,1,2,3,4,5
0,7,7,10,6,6,8
1,4,6,4,3,4,5
2,6,4,5,5,10,10
3,3,2,4,5,4,7
4,5,7,5,7,8,6


In [9]:
# Inclusive Domain Feature 
feature_names=[]
for i in range(6,12):
    feature_names.append("{}".format(i))
xx =np.array(Fec)
data2=pd.DataFrame(xx,columns=feature_names)
data2.head()

,6,7,8,9,10,11
0,27,28,28,31,26,29
1,26,33,31,28,30,28
2,24,24,24,24,24,25
3,34,32,37,36,38,44
4,32,34,33,36,34,34


In [10]:
result = pd.concat([data1,data2,label], axis=1)
result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,class
0,7,7,10,6,6,8,27,28,28,31,26,29,3
1,4,6,4,3,4,5,26,33,31,28,30,28,1
2,6,4,5,5,10,10,24,24,24,24,24,25,5
3,3,2,4,5,4,7,34,32,37,36,38,44,5
4,5,7,5,7,8,6,32,34,33,36,34,34,3


In [12]:
# Train Test Spliting
n=len(label)
import random
p=random.sample(range(Number_nodes), 1000)
#print(p)
q=[]
for i in range(n):
    if i not in p:
        q.append(i)

In [13]:
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
feature=[]
for i in range(12):
    feature.append("{}".format(i))
#print(feature)
X=result[feature] # Features
y=result['class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)  
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")

Accuracy: 90.9 %
